# Arbitrage Events Scraper

This script will look for statistical arbitrage opportunities in the upcoming eight games across all sports using Live Sports Odds API.

Documentation Link: https://the-odds-api.com/ 

### Importing Libraries

In [59]:
import requests
import json
import xlsxwriter
import pandas as pd
import numpy as np
import openpyxl
import re
from unidecode import unidecode
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Border, Side, Font, Alignment, PatternFill, numbers

### Acquiring unique API_KEY  

This key is unique. Retrieve one at TheOddsAPI website.

In [60]:
API_KEY = '9decd631fa6303329122863cffa97744'

### Defining Constants and Making API Pull

- ``BET_SIZE`` is the monetary amount in USD that you are willing to make for each bet. Defining ``BET_SIZE`` as 100 is telling the program that you want to bet a total of 100 dollars for each arbitrage opportunity that the program finds.

In [61]:
SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

BET_SIZE = 100

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
).json()

In [62]:
# odds_response
print(odds_response)

# saves data to upcoming_events.txt
with open('1_upcoming_events.txt', 'w') as f:
    json.dump(odds_response, f)

[{'id': '56f790ad0d89b1da2321001b64be5498', 'sport_key': 'baseball_mlb', 'sport_title': 'MLB', 'commence_time': '2023-06-05T22:40:00Z', 'home_team': 'Philadelphia Phillies', 'away_team': 'Detroit Tigers', 'bookmakers': [{'key': 'bovada', 'title': 'Bovada', 'last_update': '2023-06-06T01:07:55Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:55Z', 'outcomes': [{'name': 'Detroit Tigers', 'price': 15.0}, {'name': 'Philadelphia Phillies', 'price': 1.02}]}]}, {'key': 'pointsbetus', 'title': 'PointsBet (US)', 'last_update': '2023-06-06T01:07:38Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:38Z', 'outcomes': [{'name': 'Detroit Tigers', 'price': 46.0}, {'name': 'Philadelphia Phillies', 'price': 1.0}]}]}, {'key': 'betmgm', 'title': 'BetMGM', 'last_update': '2023-06-06T01:07:12Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:12Z', 'outcomes': [{'name': 'Detroit Tigers', 'price': 34.0}, {'name': 'Philadelphia Phillies', 'price': 1.01}]}]}]}, {'id': '3b

### Event Class


The Event class represents a sporting event and provides methods to find the best odds, determine if there is an opportunity for arbitrage, convert decimal odds to American odds, and calculate the bet amounts for arbitrage betting.

The code is structured in a way that encapsulates the functionality related to the event and its odds. It utilizes the data provided in the constructor to initialize the event and extract relevant information such as the sport key and ID.

- find_best_odds iterates over the bookmakers and outcomes to find the best odds for each outcome in the event. It populates the best_odds list with the bookmaker, outcome name, and odds.

- arbitrage calculates the total arbitrage percentage based on the best odds and determines if there is an opportunity for arbitrage betting. It checks if the sum of the reciprocals of the odds is less than 1.

- convert_decimal_to_american converts the decimal odds in best_odds to American odds format.

- calculate_arbitrage_bets calculates the bet amounts for each outcome based on the individual arbitrage percentages and the total arbitrage percentage.

- Each ``Event`` object represents an indivudal sporting event.

- The ``data`` parameter contains all of the odds data that is received from the API call.

In [63]:
BOOKMAKER_INDEX = 0  # Index for the bookmaker in the best_odds list
NAME_INDEX = 1  # Index for the name in the best_odds list
ODDS_INDEX = 2  # Index for the odds in the best_odds list
FIRST = 0  # Constant value for the first element in a list

class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']  # Store the sport_key from the data dictionary
        self.id = data['id']  # Store the id from the data dictionary
        
    def find_best_odds(self):
        # Determine the number of possible outcomes for a sporting event
        num_outcomes = len(self.data['bookmakers'][FIRST]['markets'][FIRST]['outcomes'])
        self.num_outcomes = num_outcomes

        # Initialize the best_odds list with initial values
        best_odds = [[None, None, float('-inf')] for _ in range(num_outcomes)]
        # Format: [Bookmaker, Name, Price]

        bookmakers = event.data['bookmakers']  # Retrieve the bookmakers from the event data
        for index, bookmaker in enumerate(bookmakers):

            # Determine the odds offered by each bookmaker
            for outcome in range(num_outcomes):

                # Check if any of the bookmaker odds are better than the current best odds
                bookmaker_odds = float(bookmaker['markets'][FIRST]['outcomes'][outcome]['price'])
                current_best_odds = best_odds[outcome][ODDS_INDEX]

                if bookmaker_odds > current_best_odds:
                    best_odds[outcome][BOOKMAKER_INDEX] = bookmaker['title']
                    best_odds[outcome][NAME_INDEX] = bookmaker['markets'][FIRST]['outcomes'][outcome]['name']
                    best_odds[outcome][ODDS_INDEX] = bookmaker_odds
                    
        self.best_odds = best_odds
        return best_odds
    
    def arbitrage(self):
        total_arbitrage_percentage = 0
        for odds in self.best_odds:
            total_arbitrage_percentage += (1.0 / odds[ODDS_INDEX])
            
        self.total_arbitrage_percentage = total_arbitrage_percentage
        self.expected_earnings = (BET_SIZE / total_arbitrage_percentage) - BET_SIZE
        
        # Check if the sum of the reciprocals of the odds is less than 1, indicating an arbitrage opportunity
        if total_arbitrage_percentage < 1:
            return True
        return False
    
    # Convert decimal/European best odds to American best odds
    def convert_decimal_to_american(self):
        best_odds = self.best_odds
        for odds in best_odds:
            decimal = odds[ODDS_INDEX]
            if decimal >= 2:
                american = (decimal - 1) * 100
            elif decimal < 2:
                american = -100 / (decimal - 1)
            odds[ODDS_INDEX] = round(american, 2)
        return best_odds
     
    def calculate_arbitrage_bets(self):
        bet_amounts = []
        for outcome in range(self.num_outcomes):
            individual_arbitrage_percentage = 1 / self.best_odds[outcome][ODDS_INDEX]
            bet_amount = (BET_SIZE * individual_arbitrage_percentage) / self.total_arbitrage_percentage
            bet_amounts.append(round(bet_amount, 2))
        
        self.bet_amounts = bet_amounts
        return


### Parsing Events and Calculating Arbitrage Opportunities

First loop iterates over the odds_response data and creates an Event object for each data item. The Event objects are then added to the events list.

Second loop iterates over the events list and checks if each event has an opportunity for arbitrage by calling the arbitrage() function. If an event has an opportunity for arbitrage, it is added to the arbitrage_events list.

Last loop iterates over the arbitrage_events list and performs calculations related to arbitrage betting for each event. The calculate_arbitrage_bets() method calculates the bet amounts, while the convert_decimal_to_american() method converts the odds from decimal format to American format.

This calculation will use unbiased arbitrage, where the profit is the same regardless of the outcome.

In [64]:
events = []  # Create an empty list to store events

for data in odds_response:
    events.append(Event(data))  # Create an Event object for each data item and add it to the events list
    print(data) 

arbitrage_events = []  # Create an empty list to store arbitrage events

for event in events:
    best_odds = event.find_best_odds()  # Find the best odds for the event
    if event.arbitrage():  # Check if the event presents an arbitrage opportunity
        arbitrage_events.append(event)  # If so, add the event to the arbitrage_events list

for event in arbitrage_events:
    event.calculate_arbitrage_bets()  # Calculate the bet amounts for the event's arbitrage bets
    event.convert_decimal_to_american()  # Convert the decimal odds to American odds



{'id': '56f790ad0d89b1da2321001b64be5498', 'sport_key': 'baseball_mlb', 'sport_title': 'MLB', 'commence_time': '2023-06-05T22:40:00Z', 'home_team': 'Philadelphia Phillies', 'away_team': 'Detroit Tigers', 'bookmakers': [{'key': 'bovada', 'title': 'Bovada', 'last_update': '2023-06-06T01:07:55Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:55Z', 'outcomes': [{'name': 'Detroit Tigers', 'price': 15.0}, {'name': 'Philadelphia Phillies', 'price': 1.02}]}]}, {'key': 'pointsbetus', 'title': 'PointsBet (US)', 'last_update': '2023-06-06T01:07:38Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:38Z', 'outcomes': [{'name': 'Detroit Tigers', 'price': 46.0}, {'name': 'Philadelphia Phillies', 'price': 1.0}]}]}, {'key': 'betmgm', 'title': 'BetMGM', 'last_update': '2023-06-06T01:07:12Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:12Z', 'outcomes': [{'name': 'Detroit Tigers', 'price': 34.0}, {'name': 'Philadelphia Phillies', 'price': 1.01}]}]}]}
{'id': '3bc3

In [65]:
with open('2_arbitrage_events.txt', 'w') as file:
    for event in arbitrage_events:
        event.calculate_arbitrage_bets()
        event.convert_decimal_to_american()
        file.write(str(event.data) + '\n')
        print(event.data)

{'id': '51202dd62c556f420b6904eec64aba66', 'sport_key': 'baseball_mlb', 'sport_title': 'MLB', 'commence_time': '2023-06-05T23:05:00Z', 'home_team': 'Pittsburgh Pirates', 'away_team': 'Oakland Athletics', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-06-06T01:06:51Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:06:51Z', 'outcomes': [{'name': 'Oakland Athletics', 'price': 1.51}, {'name': 'Pittsburgh Pirates', 'price': 2.6}]}]}, {'key': 'bovada', 'title': 'Bovada', 'last_update': '2023-06-06T01:07:55Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:55Z', 'outcomes': [{'name': 'Oakland Athletics', 'price': 1.54}, {'name': 'Pittsburgh Pirates', 'price': 2.4}]}]}, {'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2023-06-06T01:07:42Z', 'markets': [{'key': 'h2h', 'last_update': '2023-06-06T01:07:42Z', 'outcomes': [{'name': 'Oakland Athletics', 'price': 1.43}, {'name': 'Pittsburgh Pirates', 'price': 2.8}]}]}, {'key': 'circaspor

### Creating Dataframe and Writing to Excel File

This part generates an Excel file that presents information about arbitrage events, including their IDs, sport keys, expected earnings, and details about each outcome's bookmaker, name, odds, and bet amount.

In [66]:
try:
    MAX_OUTCOMES = max([event.num_outcomes for event in arbitrage_events])  # Find the maximum number of outcomes among all arbitrage events
except ValueError:
    print("No arbitrage events found.")  # If there are no arbitrage events, print a message
    MAX_OUTCOMES = 0

ARBITRAGE_EVENTS_COUNT = len(arbitrage_events)  # Count the number of arbitrage events

my_columns = [
    'ID',
    'Sport Key',
    'Expected Earnings'
] + list(
    np.array([[f'Bookmaker #{outcome}', f'Name #{outcome}', f'Odds #{outcome}', f'Amount to Buy #{outcome}'] for outcome in range(1, MAX_OUTCOMES + 1)]).flatten()
)  # Create a list of column names for the DataFrame

dataframe = pd.DataFrame(columns=my_columns)  # Create an empty DataFrame with the specified column names

for event in arbitrage_events:
    print(event.best_odds)
    row = [
        event.id,
        event.sport_key,
        round(event.expected_earnings, 2)
    ]  # Create a row with the event's ID, sport key, and rounded expected earnings

    for index, outcome in enumerate(event.best_odds):
        row += [
            outcome[BOOKMAKER_INDEX],
            outcome[NAME_INDEX],
            outcome[ODDS_INDEX],
            event.bet_amounts[index]
        ]  # Add bookmaker, name, odds, and bet amount for each outcome to the row

    while len(row) < len(my_columns):
        row.append('N/A')  # Add 'N/A' for any remaining columns (if there are fewer outcomes than the maximum)

    dataframe.loc[len(dataframe.index)] = row  # Add the row to the DataFrame

with pd.ExcelWriter('3_arbitrage_bets.xlsx') as writer:
    dataframe.to_excel(writer, index=False)  # Write the DataFrame to an Excel file, excluding the index


[['Circa Sports', 'Oakland Athletics', 0.57], ['FanDuel', 'Pittsburgh Pirates', 17900.0]]
[['PointsBet (US)', 'Cincinnati Reds', 0.2], ['FanDuel', 'Milwaukee Brewers', 53900.0]]
[['FanDuel', 'St. Louis Cardinals', 35900.0], ['Circa Sports', 'Texas Rangers', 0.28]]


In [67]:
for event in arbitrage_events:
    row = []  # Create an empty list to store the values for each row
    row.append(event.id)  # Append the event id to the row list
    row.append(event.sport_key)  # Append the sport key to the row list
    row.append(round(event.expected_earnings, 2))  # Append the rounded expected earnings to the row list

    for index, outcome in enumerate(event.best_odds):
        # Append the bookmaker, name, odds, and bet amount for each outcome to the row list
        row.append(outcome[BOOKMAKER_INDEX])
        row.append(outcome[NAME_INDEX])
        row.append(outcome[ODDS_INDEX])
        row.append(event.bet_amounts[index])

    # Add 'N/A' values to the row list to match the length of the dataframe columns
    while len(row) < len(dataframe.columns):
        row.append('N/A')

    dataframe.loc[len(dataframe.index)] = row  # Add the row to the dataframe

In [68]:
with pd.ExcelWriter('3_arbitrage_bets.xlsx') as writer:
    dataframe.to_excel(writer, index=False)

### Formatting the Excel File

This part of the code is responsible for applying formatting and styling to the cells in the Excel worksheet.

In [69]:
BLACK = '000000'  # Define color constant for black
LIGHT_GREY = 'D6D6D6'  # Define color constant for light grey
DARK_GREY = '9F9F9F'  # Define color constant for dark grey
RED = 'FEA0A0'  # Define color constant for red
BLUE = 'A0CEFE'  # Define color constant for blue
YELLOW = 'FFE540'  # Define color constant for yellow

COLORS = [RED, BLUE]  # Create a list of colors

ID_COLUMN_FILL = PatternFill(fill_type='solid', start_color=DARK_GREY, end_color=DARK_GREY)  # Create fill pattern for ID column
SPORT_KEY_COLUMN_FILL = PatternFill(fill_type='solid', start_color=LIGHT_GREY, end_color=LIGHT_GREY)  # Create fill pattern for sport key column
EXPECTED_EARNINGS_COLUMN_FILL = PatternFill(fill_type='solid', start_color=YELLOW, end_color=YELLOW)  # Create fill pattern for expected earnings column

CENTER_ALIGNMENT = Alignment(horizontal='center', vertical='bottom', indent=0)  # Define alignment style for center alignment

TOP_ROW_BORDER = Border(bottom=Side(border_style='thick', color=BLACK))  # Define border style for top row
NORMAL_ROW_BORDER = Border(top=Side(border_style='thin', color=LIGHT_GREY), bottom=Side(border_style='thin', color=DARK_GREY))  # Define border style for normal rows

wb = load_workbook('3_arbitrage_bets.xlsx')  # Load the 'bets.xlsx' workbook
ws = wb.active  # Select the active worksheet
ws.title = 'Upcoming'  # Set the title of the worksheet to 'Upcoming'

# Changing width of columns
for col in range(1, 26):
    ws.column_dimensions[chr(col + 64)].width = 20  # Set the width of each column

# Apply fill and alignment styles to cells in column A for ID
for cell in ws['A']:
    cell.fill = ID_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT

# Apply fill and alignment styles to cells in column B for sport key
for cell in ws['B']:
    cell.fill = SPORT_KEY_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT

# Apply fill, alignment, and number format styles to cells in column C for expected earnings
for cell in ws['C']:
    cell.fill = EXPECTED_EARNINGS_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    cell.number_format = numbers.BUILTIN_FORMATS[7]

START_INDEX = 'D'
# Apply alternating fill colors, alignment styles, and number formats to groups of four columns
for index in range(MAX_OUTCOMES):
    for col in ws[START_INDEX:chr(ord(START_INDEX) + 3)]:
        for cell in col:
            color = COLORS[int(index % 2)]
            cell.fill = PatternFill(fill_type='solid', start_color=color, end_color=color)
            cell.alignment = CENTER_ALIGNMENT
            if cell.column % 4 == 3:
                cell.number_format = numbers.BUILTIN_FORMATS[7]

    START_INDEX = chr(ord(START_INDEX) + 4)

# Apply border style to the top row of the worksheet
for cell in ws['1']:
    cell.border = TOP_ROW_BORDER

# Apply border style to normal rows
for row in range(2, ARBITRAGE_EVENTS_COUNT + 2):
    for cell in ws[str(row)]:
        cell.border = NORMAL_ROW_BORDER

wb.save('3_arbitrage_bets.xlsx')  # Save the modified workbook
